In [ ]:
from pyarrow import fs
import pyarrow.dataset as ds
import pyarrow as pa
import s3fs
import pandas as pd

# Configuração do ambiente

Variáveis:
- FILENAME: Nome do arquivo que deve ser lido do raw e armazenado no parquet. NUNCA FAZER MODIFICACOES NO RAW
- config: Dicionários com credenciais.

In [ ]:
FILENAME = 'base-ceb-15-04-2025-utf8'
SOURCE_PATH = 'raw/UCTIS/bases-utf8'
TARGET_PATH='UCTIS/bases/bases-utf8/base-ceb-15-04-2025'
RA = 'aguas-claras-av-araucarias'
RESPONSAVEL_NOME = 'matheus.fernandes'
RESPONSAVEL_SETOR = 'UCTIS/COGEI/GEINO'
RESPONSAVEL_EMAIL = 'matheus.fernandes@ipe.df.gov.br'
PROCESSED = 'false'

In [ ]:
minio = s3fs.S3FileSystem(key="", # change default key 
                        secret="", # change default secret
                        use_ssl=False,
                        client_kwargs={
                            'endpoint_url': 'http://minio:9000'} # change ip
                         )
minio.ls(SOURCE_PATH)

In [ ]:
# Le arquivo csv do minio
with minio.open(f'{SOURCE_PATH}/{FILENAME}.csv', 'r', encoding='utf-8') as f:
    response = f.read()

# Copia para um dataframe
# response
from io import BytesIO, StringIO
df = pd.read_csv(StringIO(response), sep=';')
df

In [ ]:
df_copy = df.copy()

# Filtragem

Realiza a filtragem por RA

In [ ]:
df_copy = df.loc[(df_copy['ra'] == '?üguas Claras') & (df_copy['classe'] == 'RESIDENCIAL')]
df_copy

In [ ]:
df2 = df_copy

# Extração

Extrai apenas as colunas que tem a ver com endereço e filtra apenas as quadras fora de nucleo rural.


In [ ]:
import re
pattern = re.compile(r'^(AV ARAUCARIAS)')
df2 = df2[
    ['endereco', 'latitude', 'longitude', 'localidade', 'classe', 'bairro_pda']
]
df2 = df2.loc[
    df2['endereco'].str.contains(pattern)
]
# df2['endereco'] = df2['endereco'].str.replace('H I', 'H/I').str.replace('H /I', 'H/I').str.replace('H E I', 'H/I')
df2

# Tratamento dos dados
Remoção de valores nulos e quebra da coluna de endereço nos seguintes componentes:

- identificador_setor (ST, setor);
- setor;
- identificador_quadra (QD, quadra);
- quadra;
- identificador_conjunto (CJ, conjunto, BL, bloco);
- conjunto;
- identificador_lote (LT, lote, CS, casa);
- lote;
- complemento (opcional). Ex: nome do edifício;
- regiao_administrativa,


In [ ]:
# Quebra da coluna de endereco nos componentes acima
df2['identificador_setor'] = df2['endereco'].str.split(' ').str[0]
df2['setor'] = df2['endereco'].str.split(' ').str[1]

df2['identificador_quadra'] = df2['endereco'].str.extract(r'.*(LT).*')
df2['quadra'] = df2['endereco'].str.extract(r'LT (\S+) \S+')

df2['identificador_conjunto'] = df2['endereco'].str.extract(r'.*(BL).*')
df2['conjunto'] = df2['endereco'].str.extract(r'BL ([A-Z]) \S+')

df2['identificador_lote'] = df2['endereco'].str.extract(r'.*([AS][PL]).*') # Match "SL" or "AP"
df2['lote'] = df2['endereco'].str.extract(r'[AS][PL] (\d+) ')

# Renomeia colunas
# df2.rename(columns={'bairro_pda': 'setor'}, inplace=True)

df2

In [ ]:
# Remove colunas
del df2['endereco']

# Carregamento

Carregamento no bucket `intermediate` para análise com outras ferramentas ou mais tratamento para carregamento no DW.

In [ ]:
# Converte para parquet e sobe para o minio
from pyarrow import Table, parquet as pq
minio_url = f's3://intermediate/{TARGET_PATH}/{FILENAME}-{RA}.parquet'

pq.write_table(
    Table.from_pandas(df2),
    minio_url,
    filesystem=minio,
    use_dictionary=True,
    compression='snappy',
    version='2.6',
)

In [ ]:
# Adiciona tags ao arquivo
tags = {
    'User': RESPONSAVEL_NOME,
    'Department': RESPONSAVEL_SETOR,
    'Email': RESPONSAVEL_EMAIL,
    'Processed': PROCESSED,
}
parquet_partitions = minio.ls(f'intermediate/{TARGET_PATH}/{FILENAME}-{RA}.parquet')
# parquet_partitions
for p in parquet_partitions:
    minio.put_tags(p, tags, mode='o')